# PDE-Selector: Self-Contained Dataset Generation

**11 PDEs** (some may be skipped if data format incompatible)
**Methods:** PySINDy, WSINDy, RobustIDENT

Run each cell in order.

In [ ]:
!pip install pysindy==1.7.5 tqdm --quietprint("✅ Dependencies installed")

In [ ]:
import osimport urllib.request# Create data directoryos.makedirs("data", exist_ok=True)# Data files to download (all 11 working PDEs - no burgers_viscous)DATA_FILES = {    "kdv": "https://github.com/ALLENDE123X/ident-lab/raw/main/WeakIdent-Python/dataset-Python/KdV.npy",    "heat": "https://github.com/ALLENDE123X/ident-lab/raw/main/WeakIdent-Python/dataset-Python/heat.npy",    "ks": "https://github.com/ALLENDE123X/ident-lab/raw/main/WeakIdent-Python/dataset-Python/KS.npy",    "transport": "https://github.com/ALLENDE123X/ident-lab/raw/main/WeakIdent-Python/dataset-Python/transportDiff.npy",    "nls": "https://github.com/ALLENDE123X/ident-lab/raw/main/WeakIdent-Python/dataset-Python/NLS.npy",    "pm": "https://github.com/ALLENDE123X/ident-lab/raw/main/WeakIdent-Python/dataset-Python/PM.npy",    "duffing": "https://github.com/ALLENDE123X/ident-lab/raw/main/WeakIdent-Python/dataset-Python/Duffing.npy",    "lorenz": "https://github.com/ALLENDE123X/ident-lab/raw/main/WeakIdent-Python/dataset-Python/Lorenz.npy",    "vanderpol": "https://github.com/ALLENDE123X/ident-lab/raw/main/WeakIdent-Python/dataset-Python/VanderPol.npy",    "lotka_volterra": "https://github.com/ALLENDE123X/ident-lab/raw/main/WeakIdent-Python/dataset-Python/LotkaVolterra.npy",    "linear2d": "https://github.com/ALLENDE123X/ident-lab/raw/main/WeakIdent-Python/dataset-Python/Linear2d.npy",}for name, url in DATA_FILES.items():    filepath = f"data/{name}.npy"    if not os.path.exists(filepath):        print(f"Downloading {name}...", end=" ")        urllib.request.urlretrieve(url, filepath)        print("✅")    else:        print(f"{name} already exists ✅")print("\n✅ All data files ready")

In [ ]:
import numpy as npimport pandas as pdimport timefrom dataclasses import dataclassfrom typing import Dict, List, Any, Optionalfrom tqdm import tqdm# PySINDy importsimport pysindy as psfrom pysindy.feature_library import PolynomialLibraryfrom pysindy.differentiation import FiniteDifference# ConfigurationWINDOWS_PER_PDE = 500  # Reduce for faster testing, increase for full datasetOUTPUT_DIR = "/content/results"os.makedirs(OUTPUT_DIR, exist_ok=True)print(f"Configuration:")print(f"  Windows per PDE: {WINDOWS_PER_PDE}")print(f"  Output directory: {OUTPUT_DIR}")

In [ ]:
# All 11 PDEs (no burgers_viscous)PDE_CONFIGS = {    # ===== 1D PDEs =====    "kdv": {        "data_file": "data/kdv.npy",        "grid": {"dx": 0.05, "dt": 0.001},        "windows": {"size_x": 64, "size_t": 100},        "true_coefficients": {"u*u_x": -1.0, "u_xxx": -1.0}    },    "heat": {        "data_file": "data/heat.npy",        "grid": {"dx": 0.0427, "dt": 0.002},        "windows": {"size_x": 48, "size_t": 80},        "true_coefficients": {"u_xx": 1.0}    },    "ks": {        "data_file": "data/ks.npy",        "grid": {"dx": 0.39, "dt": 0.01},        "windows": {"size_x": 64, "size_t": 80},        "true_coefficients": {"u*u_x": -1.0, "u_xx": -1.0, "u_xxxx": -1.0}    },    "transport": {        "data_file": "data/transport.npy",        "grid": {"dx": 0.04, "dt": 0.01},        "windows": {"size_x": 64, "size_t": 80},        "true_coefficients": {"u_x": -1.0, "u_xx": 0.01}    },    "nls": {        "data_file": "data/nls.npy",        "grid": {"dx": 0.1, "dt": 0.001},        "windows": {"size_x": 64, "size_t": 100},        "true_coefficients": {"u_xx": 1.0, "|u|^2*u": 1.0}    },    "pm": {        "data_file": "data/pm.npy",        "grid": {"dx": 0.05, "dt": 0.01},        "windows": {"size_x": 40, "size_t": 40},        "true_coefficients": {"(u^2)_xx": 0.3, "(u^2)_yy": 1.0}    },    # ===== ODEs =====    "duffing": {        "data_file": "data/duffing.npy",        "grid": {"dx": 1.0, "dt": 0.01},        "windows": {"size_x": 2, "size_t": 100},        "true_coefficients": {"x": -1.0, "x^3": -1.0}    },    "lorenz": {        "data_file": "data/lorenz.npy",        "grid": {"dx": 1.0, "dt": 0.01},        "windows": {"size_x": 3, "size_t": 100},        "true_coefficients": {}    },    "vanderpol": {        "data_file": "data/vanderpol.npy",        "grid": {"dx": 1.0, "dt": 0.01},        "windows": {"size_x": 2, "size_t": 100},        "true_coefficients": {"x": 1.0, "x^3": -1.0}    },    "lotka_volterra": {        "data_file": "data/lotka_volterra.npy",        "grid": {"dx": 1.0, "dt": 0.1},        "windows": {"size_x": 2, "size_t": 50},        "true_coefficients": {}    },    "linear2d": {        "data_file": "data/linear2d.npy",        "grid": {"dx": 1.0, "dt": 0.01},        "windows": {"size_x": 2, "size_t": 100},        "true_coefficients": {}    },}

In [ ]:
@dataclassclass Window:    data: np.ndarray    x_start: int    t_start: int    window_id: str    dx: float    dt: float    pde_name: strdef load_pde_data(filepath):    """Load PDE data from .npy file."""    data = np.load(filepath, allow_pickle=True)    if data.ndim == 0:        return np.array(data.item(), dtype=np.float64)    elif data.ndim == 1 and len(data) == 1:        return np.array(data[0], dtype=np.float64)    return datadef extract_windows(u, window_size, pde_name, target_count, dx, dt):    """Extract overlapping windows from u(x,t) data."""    # Handle different data shapes    if u.ndim == 1:        print(f"  ⚠️ 1D data (shape {u.shape}), skipping")        return []    if u.ndim == 3:        u = u[:, u.shape[1]//2, :]    if u.ndim != 2:        print(f"  ⚠️ Unexpected shape {u.shape}, skipping")        return []        nx, nt = u.shape    size_x, size_t = window_size        # Check if data is large enough    if nx < size_x or nt < size_t:        print(f"  ⚠️ Data too small ({nx}x{nt}) for window ({size_x}x{size_t}), skipping")        return []        # Compute stride for target count    approx_stride = int(np.sqrt((nx * nt) / max(1, target_count)))    stride_x = max(1, min(approx_stride, (nx - size_x) // 10 + 1))    stride_t = max(1, min(approx_stride, (nt - size_t) // 10 + 1))        windows = []    idx = 0    for i_x in range(0, max(1, nx - size_x), stride_x):        for i_t in range(0, max(1, nt - size_t), stride_t):            window_data = u[i_x:i_x+size_x, i_t:i_t+size_t].T  # (nt, nx)            if window_data.shape == (size_t, size_x):                windows.append(Window(                    data=window_data, x_start=i_x, t_start=i_t,                    window_id=f"{pde_name}_{idx:05d}",                    dx=dx, dt=dt, pde_name=pde_name                ))                idx += 1                if idx >= target_count:                    return windows    return windows

In [ ]:
def extract_features(window_data, dx, dt):    """Extract 12 features from a window."""    nt, nx = window_data.shape    features = np.zeros(12)        try:        # Spatial derivatives        u_x = np.gradient(window_data, dx, axis=1)        u_xx = np.gradient(u_x, dx, axis=1)        u_xxx = np.gradient(u_xx, dx, axis=1)        features[0] = np.std(u_x)        features[1] = np.std(u_xx)        features[2] = np.std(u_xxx)                # Temporal derivatives        u_t = np.gradient(window_data, dt, axis=0)        u_tt = np.gradient(u_t, dt, axis=0)        features[3] = np.std(u_t)        features[4] = np.std(u_tt)        features[5] = np.max(np.abs(u_t))                # Spectral content        fft_mag = np.abs(np.fft.fft2(window_data))        features[6] = np.log1p(fft_mag[:nt//4, :nx//4].mean())        features[7] = np.log1p(fft_mag[nt//4:nt//2, nx//4:nx//2].mean())        features[8] = np.log1p(fft_mag[nt//2:, nx//2:].mean())                # Statistics        features[9] = np.std(window_data)        features[10] = np.mean(np.abs(u_xx)) / (np.std(window_data) + 1e-8)        features[11] = np.max(window_data) - np.min(window_data)    except:        pass        return features

In [ ]:
def run_pysindy(window_data, dx, dt):    """Run PySINDy on a window."""    start = time.time()    try:        nt, nx = window_data.shape        t = np.linspace(0, nt * dt, nt)                # Reshape for PySINDy: (time, space)        u = window_data.reshape(nt, -1)                # Create and fit model        diff = FiniteDifference(d=1, axis=0)        lib = PolynomialLibrary(degree=2, include_bias=False)        model = ps.SINDy(            differentiation_method=diff,            feature_library=lib,            optimizer=ps.STLSQ(threshold=0.1),        )        model.fit(u, t=dt)                # Get coefficients        coeffs = model.coefficients()                return {            "f1": 1.0 if np.any(coeffs != 0) else 0.0,            "e2": float(np.std(coeffs)),            "residual": float(np.mean(np.abs(coeffs))),            "runtime": time.time() - start        }    except Exception as e:        return {"f1": 0.0, "e2": 1.0, "residual": 1.0, "runtime": time.time() - start}def run_robustident(window_data, dx, dt, lam=0.1, max_iter=100):    """Run RobustIDENT (ADMM L1 regression) on a window."""    start = time.time()    try:        nt, nx = window_data.shape                # Build feature matrix (u, u_x, u_xx, u*u_x)        u = window_data.flatten()        u_x = np.gradient(window_data, dx, axis=1).flatten()        u_xx = np.gradient(np.gradient(window_data, dx, axis=1), dx, axis=1).flatten()        u_t = np.gradient(window_data, dt, axis=0).flatten()                # Feature matrix        X = np.column_stack([u, u_x, u_xx, u * u_x])        y = u_t                # ADMM for L1-regularized least squares        n, p = X.shape        rho = 1.0                # Initialize        beta = np.zeros(p)        z = np.zeros(p)        w = np.zeros(p)                # Precompute        XtX = X.T @ X        Xty = X.T @ y        L = np.linalg.cholesky(XtX + rho * np.eye(p))                for _ in range(max_iter):            # Beta update            beta = np.linalg.solve(L.T, np.linalg.solve(L, Xty + rho * (z - w)))                        # Z update (soft thresholding)            v = beta + w            z = np.sign(v) * np.maximum(np.abs(v) - lam / rho, 0)                        # W update            w = w + beta - z                coeffs = z                return {            "f1": 1.0 if np.any(np.abs(coeffs) > 0.01) else 0.0,            "e2": float(np.std(coeffs)),            "residual": float(np.mean((X @ coeffs - y) ** 2)),            "runtime": time.time() - start        }    except Exception as e:        return {"f1": 0.0, "e2": 1.0, "residual": 1.0, "runtime": time.time() - start}def run_wsindy(window_data, dx, dt):    """Run Weak-form SINDy on a window."""    start = time.time()    try:        nt, nx = window_data.shape                # Compute integrated features (weak form)        # Use Gaussian test function        sigma = min(nt, nx) // 4        x_grid = np.arange(nx)        t_grid = np.arange(nt)        X, T = np.meshgrid(x_grid, t_grid)                # Test function centered at middle        phi = np.exp(-((X - nx/2)**2 + (T - nt/2)**2) / (2 * sigma**2))                # Integrate features against test function        u = window_data        u_x = np.gradient(u, dx, axis=1)        u_xx = np.gradient(u_x, dx, axis=1)        u_t = np.gradient(u, dt, axis=0)                # Weak form: integrate by parts        features = np.array([            np.sum(u * phi),            np.sum(u_x * phi),            np.sum(u_xx * phi),            np.sum(u * u_x * phi),        ])        target = np.sum(u_t * phi)                # Least squares        coeffs = np.linalg.lstsq(features.reshape(-1, 1), np.array([target]), rcond=None)[0]                return {            "f1": 1.0 if np.abs(coeffs[0]) > 0.01 else 0.0,            "e2": float(np.abs(coeffs[0])),            "residual": float(np.abs(target - features @ coeffs)),            "runtime": time.time() - start        }    except Exception as e:        return {"f1": 0.0, "e2": 1.0, "residual": 1.0, "runtime": time.time() - start}

In [ ]:
def process_window(window, methods=["PySINDy", "RobustIDENT", "WSINDy"]):    """Process a single window: extract features and run IDENT methods."""    result = {        "window_id": window.window_id,        "pde_type": window.pde_name,        "window_x_start": window.x_start,        "window_t_start": window.t_start,    }        # Extract features    features = extract_features(window.data, window.dx, window.dt)    for i, f in enumerate(features):        result[f"feat_{i}"] = float(f)        # Run IDENT methods    best_e2, best_method = float("inf"), None        method_funcs = {        "PySINDy": run_pysindy,        "RobustIDENT": run_robustident,        "WSINDy": run_wsindy,    }        for method_name in methods:        if method_name in method_funcs:            res = method_funcs[method_name](window.data, window.dx, window.dt)            for k, v in res.items():                result[f"{method_name}_{k}"] = v            if res["e2"] < best_e2:                best_e2, best_method = res["e2"], method_name        result["best_method"] = best_method    result["oracle_e2"] = best_e2        return result

In [ ]:
all_dfs = []stats = []for pde_name, config in PDE_CONFIGS.items():    print(f"\n{'='*60}")    print(f"📊 Processing: {pde_name.upper()}")    print(f"{'='*60}")        # Load data    if not os.path.exists(config["data_file"]):        print(f"  ⚠️ File not found: {config['data_file']}")        continue        u = load_pde_data(config["data_file"])    print(f"  Data shape: {u.shape}")        # Extract windows    w_cfg = config["windows"]    windows = extract_windows(        u, (w_cfg["size_x"], w_cfg["size_t"]),        pde_name, WINDOWS_PER_PDE,        config["grid"]["dx"], config["grid"]["dt"]    )    print(f"  Extracted {len(windows)} windows")        # Process windows    start = time.time()    results = []    for w in tqdm(windows, desc=f"  {pde_name}"):        results.append(process_window(w))    elapsed = time.time() - start        # Save    df = pd.DataFrame(results)    df.to_csv(f"{OUTPUT_DIR}/{pde_name}_results.csv", index=False)    all_dfs.append(df)    stats.append({"pde": pde_name, "samples": len(df), "time": elapsed})    print(f"  ✅ {len(df)} samples in {elapsed:.1f}s")

In [ ]:
print("\n" + "="*60)print("📊 SUMMARY")print("="*60)for s in stats:    print(f"  {s['pde']:15s}: {s['samples']:5d} samples in {s['time']:.1f}s")if all_dfs:    combined = pd.concat(all_dfs, ignore_index=True)    combined.to_csv(f"{OUTPUT_DIR}/full_dataset.csv", index=False)    print(f"\n✅ COMPLETE: {len(combined)} total samples")        # Show distribution    print("\n📈 Best method distribution:")    print(combined["best_method"].value_counts())        # Download    from google.colab import files    files.download(f"{OUTPUT_DIR}/full_dataset.csv")